# Import modules

In [1]:
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import time
from datetime import timedelta
from IPython.display import display, Markdown

# Sort scattered photons from ASCII version of phase-space file

In [3]:
start_time = time.time()

E_max = 0.3
E_min = E_max/(1+2*E_max/0.511)

E_scat = []         # Energy 
x_scat = []         # x-coordinate
y_scat = []         # y-coordinate
z_scat = []         # z-coordinate
xcos_scat = []      # direction cosine wrt x-axis
ycos_scat = []      # direction cosine wrt y-axis

with open(r'D:/IAEAphsp-to-ASCII/iaea_phsp/iaea2ascii.txt','r') as f:
    for line in f:
        if float(line.split(None, 10)[8]) >= E_min and float(line.split(None, 10)[8]) < E_max:
            E_scat.append(line.split(None, 10)[8])
            x_scat.append(line.split(None, 10)[2])
            y_scat.append(line.split(None, 10)[3])
            z_scat.append(line.split(None, 10)[4])
            xcos_scat.append(line.split(None, 10)[5])
            ycos_scat.append(line.split(None, 10)[6])
            
E_scat_array = np.array(E_scat,dtype=float)
x_scat_array = np.array(x_scat,dtype=float)
y_scat_array = np.array(y_scat,dtype=float)
z_scat_array = np.array(z_scat,dtype=float)
xcos_scat_array = np.array(xcos_scat,dtype=float)
ycos_scat_array = np.array(ycos_scat,dtype=float)

# print(len(E_scat_array))
# print(len(x_scat_array))
# print(len(y_scat_array))
# print(len(z_scat_array))
# print(len(xcos_scat_array))
# print(len(ycos_scat_array))

end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 2143.857512950897 s


In [229]:
import csv

def row_count(input):
    with open(input) as f:
        for i, l in enumerate(f):
            pass
    return i

input = 'D:/IAEAphsp-to-ASCII/iaea_phsp/iaea2ascii.txt'
asd = row_count(input)

In [232]:
print(asd)

494290439


In [233]:
len(E_scat)

306096

In [19]:
len(E_scat)

323759

# Calculate central angle of scattered photons

In [4]:
start_time = time.time()

def theta(index):
    if x_scat_array[index] > 0 and y_scat_array[index] >= 0:
        return np.arctan(y_scat_array[index]/x_scat_array[index])
    if x_scat_array[index] > 0 and y_scat_array[index] <= 0:
        return np.arctan(y_scat_array[index]/x_scat_array[index]) + 2*np.pi
    if x_scat_array[index] < 0 and y_scat_array[index] >= 0:
        return (np.arctan(y_scat_array[index]/x_scat_array[index]) + np.pi)
    if x_scat_array[index] < 0 and y_scat_array[index] < 0:
        return (np.arctan(y_scat_array[index]/x_scat_array[index]) + np.pi)
    if x_scat_array[index] == 0 and y_scat_array[index] > 0:
        return np.pi/2
    if x_scat_array[index] == 0 and y_scat_array[index] < 0:
        return -np.pi/2

theta_scat_array = []
for i in range(len(x_scat_array)):
    theta_scat_array.append(theta(i))

# print(len(theta_scat_array))

end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 2.320521831512451 s


# Calculate polar angle of scattered photon's velocity w/ respect to negative y-axis

In [5]:
start_time = time.time()

def theta_dir(index):
    if xcos_scat_array[index] >= 0:
        return np.pi - np.arccos(ycos_scat_array[index])
    if xcos_scat_array[index] < 0:
        return np.pi + np.arccos(ycos_scat_array[index])

theta_dir_array = []
for i in range(len(ycos_scat_array)):
    theta_dir_array.append(theta_dir(i))

# print(len(theta_dir_array))

end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 1.6881916522979736 s


# Write (E,x,y,z,theta,theta_dir) of scattered photons to separate text file

In [6]:
start_time = time.time()

with open(r'C:/Users/Renniel/CCST_scat_kn_300.txt', 'a') as f:
    for i in range(len(E_scat_array)):
        f.write("%s,%s,%s,%s,%s,%s \n"%(E_scat_array[i],x_scat_array[i],y_scat_array[i],z_scat_array[i],theta_scat_array[i],theta_dir_array[i]))
        
end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 3.432692766189575 s


# ---------------------------------------------------------------------------------------------------------

# Load scattered photons file (E,x,y,z,theta,theta_dir)

In [11]:
start_time = time.time()

df = pd.read_csv('C:/Users/Renniel/CCST_scat_kn_300.txt',header=None)

E_array = np.asarray(df.iloc[:,0],dtype=float)
x_array = np.array(df.iloc[:,1],dtype=float)
y_array = np.array(df.iloc[:,2],dtype=float)
z_array = np.array(df.iloc[:,3],dtype=float)
theta_array = np.array(df.iloc[:,4],dtype=float)
thetadir_array = np.array(df.iloc[:,5],dtype=float)

E_scat_rnd = np.round(np.array(E_array, dtype = float), 4)

end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 0.8155348300933838 s


In [95]:
len(E_array)

371703

# Sort per detector

In [12]:
start_time = time.time()

K = 3725                       # no. of detectors
r = 47.45                      # radius of scatter ring detector
angle_int = (2*np.pi)/(K+1)    # angle covered by a detector element

scatter_det_E = [[] for k in range(K)]
scatter_det_theta = [[] for k in range(K)]
for k in range(K):
    for l in range(len(E_scat_rnd)):
        if theta_array[l] >= (angle_int/2) + k*angle_int and theta_array[l] <= (angle_int/2) + (k+1)*angle_int:
            scatter_det_E[k].append(E_scat_rnd[l])
            scatter_det_theta[k].append(theta_array[l]*180/np.pi)   # in degrees
            
end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 2149.9313271045685 s


In [209]:
len(scatter_det_E[1])

529

# Sort per detector and scattering circle ($C_1$ or $C_2$)

In [13]:
start_time = time.time()

K = 3725                       # no. of detectors
r = 47.45                      # radius of scatter ring detector
angle_int = (2*np.pi)/(K+1)    # angle covered by a detector element

scatter_det_E_c1 = [[] for k in range(K)]
scatter_det_E_c2 = [[] for k in range(K)]

for k in range(K//2+1):
    for l in range(len(E_scat_rnd)):
        if theta_array[l] >= (angle_int/2) + k*angle_int and theta_array[l] <= (angle_int/2) + (k+1)*angle_int:
            if thetadir_array[l] < np.pi*(1 + (k+1)/(K+1)):
                scatter_det_E_c1[k].append(E_scat_rnd[l])
            if thetadir_array[l] > np.pi*(1 + (k+1)/(K+1)):
                scatter_det_E_c2[k].append(E_scat_rnd[l])
                
for k in range(K//2+1, K):
    for l in range(len(E_scat_rnd)):
        if theta_array[l] >= (angle_int/2) + k*angle_int and theta_array[l] <= (angle_int/2) + (k+1)*angle_int:
            if thetadir_array[l] < np.pi*(1 + (k+1)/(K+1)) and thetadir_array[l] > np.pi:
                scatter_det_E_c1[k].append(E_scat_rnd[l])
            if thetadir_array[l] < np.pi*(1 + (k+1)/(K+1)) and thetadir_array[l] < np.pi:
                scatter_det_E_c2[k].append(E_scat_rnd[l])
            if thetadir_array[l] > np.pi*(1 + (k+1)/(K+1)):
                scatter_det_E_c2[k].append(E_scat_rnd[l])

end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 1748.640840768814 s


# Count same photon energies per detector

In [73]:
start_time = time.time()

def pcount(sc_det_E,energy_step):
    """
    sc_det_E : energy of photons per detector
    energy_step : interval or energy resolution, default (1 keV)
    """
    step = 1000/energy_step
    dec = np.floor(np.log10(np.abs(step))).astype(int)
    low = int(np.round(E_min,dec)*step)
    high = int(E_max*step)
    sc_det_E_f = [[] for k in range(K)]
    for p in range(len(sc_det_E)):
        for q in range(low,high):
            sc_det_E_f[p].append(sc_det_E[p].count(q/step))
    return sc_det_E_f
        
end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 0.0 s


In [33]:
int(np.round(E_min,3)*1000)

138

In [97]:
dec=1000/0.1
print(np.floor(np.log10(np.abs(dec))).astype(int))

4


In [54]:
1/1000.0

0.001

In [36]:
int(E_max*1000)

300

# Count no. of photons per energy per detector

In [79]:
start_time = time.time()

# Original data
scatter_det_E_f = pcount(scatter_det_E,1)

# 1st Circular Arc
scatter_det_E_f_c1 = pcount(scatter_det_E_c1,1)

# 2nd Circular Arc
scatter_det_E_f_c2 = pcount(scatter_det_E_c2,1)
        
end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 23.780762910842896 s


In [90]:
a = pd.DataFrame(scatter_det_E_f)
display(a)
print(a.max())

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
0,35,10,4,8,2,8,1,0,2,1,...,0,0,0,0,0,0,0,0,0,0
1,15,3,1,1,1,1,5,2,4,1,...,0,0,0,0,0,0,0,0,0,0
2,12,7,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,12,5,0,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,21,2,2,1,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3721,17,3,2,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3722,9,4,2,0,0,0,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
3723,17,6,2,1,1,2,1,0,2,1,...,0,0,0,0,0,0,0,0,0,0


0      62
1      25
2      13
3       9
4      10
       ..
157     5
158     6
159     5
160     7
161    10
Length: 162, dtype: int64


# Save scatter_det_E_f as csv file

In [91]:
start_time = time.time()

df_E_f = pd.DataFrame(scatter_det_E_f)
df_E_f.to_csv('C:/Users/Renniel/CTP401_CCST_kn_300_E_f.txt', index = False)
# display(df_E_f)

end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 0.4323868751525879 s


# Save scatter_det_E_f_c1 as csv file

In [92]:
start_time = time.time()

df_E_f_c1 = pd.DataFrame(scatter_det_E_f_c1)
df_E_f_c1.to_csv('C:/Users/Renniel/CTP401_CCST_kn_300_E_f_c1.txt', index = False)
# display(df_E_f_c1)

end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 0.44429802894592285 s


# Save scatter_det_E_f_c2 as csv file

In [93]:
start_time = time.time()

df_E_f_c2 = pd.DataFrame(scatter_det_E_f_c2)
df_E_f_c2.to_csv('C:/Users/Renniel/CTP401_CCST_kn_300_E_f_c2.txt', index = False)
# display(df_E_f_c2)

end_time = time.time()
print('Processing time: {}'.format(end_time - start_time), 's')

Processing time: 0.4327244758605957 s


In [225]:
display(df_E_f_c2)

,0,1,2,3,4,5,6,7,8,9,...,3825,3826,3827,3828,3829,3830,3831,3832,3833,3834
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,0,7,3,7,1,5,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
3721,0,6,4,2,6,5,1,4,2,0,...,0,0,0,0,0,0,0,0,0,0
3722,0,12,6,4,5,5,1,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3723,0,16,7,5,5,9,6,6,11,5,...,0,0,0,0,0,0,0,0,0,0
